In [23]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!pip install geopy
from geopy.geocoders import Nominatim
!pip install geocoder
import geocoder

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

import xml

!pip install folium
import folium 

print('Libraries imported.')

     |████████████████████████████████| 102kB 8.1MB/s ta 0:00:011
Libraries imported.


In [36]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Areas_of_Manchester").text

In [37]:
soup = BeautifulSoup(data, 'html.parser')

In [38]:
neighborhoodList = []

In [39]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [40]:
mn_df = pd.DataFrame({"Neighborhood": neighborhoodList})

mn_df.head()

,Neighborhood
0,Abbey Hey
1,Alport Town
2,Ancoats
3,Ardwick
4,Ardwick Green


In [41]:
mn_df.shape

(69, 1)

In [42]:
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Manchester, UK'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [43]:
coords = [ get_latlng(neighborhood) for neighborhood in mn_df["Neighborhood"].tolist() ]

In [44]:
coords

[[53.46867000000003, -2.156069999999943],
 [53.44918565614441, -2.270495460777051],
 [53.48535000000004, -2.229199999999935],
 [53.46932000000004, -2.2226199999999494],
 [53.471936341613485, -2.224624092480661],
 [53.39909000000006, -2.285609999999963],
 [53.422200974299706, -2.246092776476112],
 [42.95585287149463, -71.45901870839332],
 [53.38173000000006, -2.2612499999999613],
 [53.47839000000005, -2.2003199999999765],
 [53.51661000000007, -2.208099999999945],
 [53.47839000000005, -2.2003199999999765],
 [53.51423004848438, -2.211629896806869],
 [53.47959000000003, -2.248739999999941],
 [53.42136000000005, -2.2172799999999597],
 [53.42136000000005, -2.2172799999999597],
 [53.47626000000008, -2.2554699999999457],
 [53.491423073962316, -2.2408725491620283],
 [53.44251000000003, -2.276559999999961],
 [53.43262000000004, -2.2704499999999257],
 [53.44251000000003, -2.276559999999961],
 [53.44251000000003, -2.276559999999961],
 [53.44251000000003, -2.276559999999961],
 [53.47220000000004, -

In [45]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [46]:
mn_df['Latitude'] = df_coords['Latitude']
mn_df['Longitude'] = df_coords['Longitude']

In [47]:
print(mn_df.shape)
mn_df

(69, 3)


,Neighborhood,Latitude,Longitude
0,Abbey Hey,53.468670,-2.156070
1,Alport Town,53.449186,-2.270495
2,Ancoats,53.485350,-2.229200
3,Ardwick,53.469320,-2.222620
4,Ardwick Green,53.471936,-2.224624
5,Baguley,53.399090,-2.285610
6,Barlow Moor,53.422201,-2.246093
7,"Belle Vue, Manchester",42.955853,-71.459019
8,Benchill,53.381730,-2.261250
9,"Beswick, Manchester",53.478390,-2.200320


In [48]:
mn_df.to_csv("mn_df.csv", index=False)

In [50]:
address = 'Manchester, UK'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manchester, UK {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manchester, UK 53.4794892, -2.2451148.


In [52]:
map_mn = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, neighborhood in zip(mn_df['Latitude'], mn_df['Longitude'], mn_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mn)  
    
map_mn

In [53]:
map_mn.save('map_mn.html')

In [54]:
CLIENT_ID = 'CIK11220IV1D1P30QS5NMBP5LUVGGCEVTUJQJLNDGHOQ5POP' 
CLIENT_SECRET = 'LCTSXCT2UJQGRITVT3Y4MPWDIUXAUIJ32HF1U00PAGUNSUFX' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CIK11220IV1D1P30QS5NMBP5LUVGGCEVTUJQJLNDGHOQ5POP
CLIENT_SECRET:LCTSXCT2UJQGRITVT3Y4MPWDIUXAUIJ32HF1U00PAGUNSUFX


In [55]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mn_df['Latitude'], mn_df['Longitude'], mn_df['Neighborhood']):
    
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [56]:
venues_df = pd.DataFrame(venues)

venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4423, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abbey Hey,53.46867,-2.15607,The Gym Manchester Ashton Old Road,53.473077,-2.164137,Gym / Fitness Center
1,Abbey Hey,53.46867,-2.15607,Debdale Park,53.458288,-2.160558,Park
2,Abbey Hey,53.46867,-2.15607,PureGym,53.459749,-2.163222,Gym / Fitness Center
3,Abbey Hey,53.46867,-2.15607,Costa Coffee,53.473335,-2.164748,Coffee Shop
4,Abbey Hey,53.46867,-2.15607,London Irish Pub,53.474080,-2.148228,Pub


In [57]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Abbey Hey,39,39,39,39,39,39
Alport Town,86,86,86,86,86,86
Ancoats,100,100,100,100,100,100
Ardwick,100,100,100,100,100,100
Ardwick Green,100,100,100,100,100,100
Baguley,20,20,20,20,20,20
Barlow Moor,97,97,97,97,97,97
"Belle Vue, Manchester",100,100,100,100,100,100
Benchill,44,44,44,44,44,44


In [58]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 199 uniques categories.


In [59]:
venues_df['VenueCategory'].unique()[:50]

array(['Gym / Fitness Center', 'Park', 'Coffee Shop', 'Pub',
       'General College & University', 'Supermarket', 'Farmers Market',
       'Gymnastics Gym', 'Discount Store', 'Sandwich Place',
       'Grocery Store', 'Pizza Place', 'Fast Food Restaurant',
       'Soccer Field', 'Train Station', 'Market', 'Soccer Stadium',
       'Hotel', 'Tram Station', 'Pharmacy', 'Intersection',
       'Department Store', 'Bar', 'Deli / Bodega', 'Gastropub',
       'Cocktail Bar', 'Tapas Restaurant', 'Tea Room',
       'Japanese Restaurant', 'Food & Drink Shop', 'Turkish Restaurant',
       'Spanish Restaurant', 'Liquor Store', 'Museum', 'Nature Preserve',
       'Falafel Restaurant', 'Café', 'Restaurant', 'Fish Market',
       'Cricket Ground', 'Indian Restaurant', 'Thai Restaurant',
       'Concert Hall', 'Portuguese Restaurant', 'Electronics Store',
       'Gym', 'Fish & Chips Shop', 'Disc Golf', 'Track', 'Brewery'],
      dtype=object)

In [60]:
"Neighborhood" in venues_df['VenueCategory'].unique()

False

In [61]:
mn_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

mn_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

fixed_columns = [mn_onehot.columns[-1]] + list(mn_onehot.columns[:-1])
mn_onehot = mn_onehot[fixed_columns]

print(mn_onehot.shape)
mn_onehot.head()

(4423, 200)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Automotive Shop,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Bookstore,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Canal Lock,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Donut Shop,Duty-free Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General College & University,Go Kart Track,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Other Repair Shop,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rugby Pitch,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Track,Track Stadium,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Weight Loss Center,Wine Bar,Women's Store
0,Abbey Hey,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Abbey Hey,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Abbey Hey,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [62]:
mn_grouped = mn_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mn_grouped.shape)
mn_grouped

(69, 200)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Automotive Shop,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Bookstore,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Canal Lock,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Donut Shop,Duty-free Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General College & University,Go Kart Track,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Other Repair Shop,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rugby Pitch,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Track,Track Stadium,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Weight Loss Center,Wine Bar,Women's Store
0,Abbey Hey,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.025641,0.000000,0.00,0.000000,0.051282,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.025641,0.051282,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.025641,0.000000,0.000000,0.000000,0.051282,0.00,0.000000,0.076923,0.000000,0.025641,0.000000,0.000000,0.00,0.000000,0.00,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.051282,0.000000,0.000000,0.000000,0.025641,0.000000,0.025641,0.000000,0.000000,0.0

In [65]:
len(mn_grouped[mn_grouped["Shopping Mall"] > 0])

5

In [66]:
mn_mall = mn_grouped[["Neighborhoods","Shopping Mall"]]

In [67]:
mn_mall.head()

,Neighborhoods,Shopping Mall
0,Abbey Hey,0.0
1,Alport Town,0.0
2,Ancoats,0.0
3,Ardwick,0.0
4,Ardwick Green,0.0


In [68]:
kclusters = 3

mn_clustering = mn_mall.drop(["Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mn_clustering)

kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [69]:
mn_merged = mn_mall.copy()

mn_merged["Cluster Labels"] = kmeans.labels_

In [70]:
mn_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mn_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Abbey Hey,0.0,0
1,Alport Town,0.0,0
2,Ancoats,0.0,0
3,Ardwick,0.0,0
4,Ardwick Green,0.0,0


In [71]:
mn_merged = mn_merged.join(mn_df.set_index("Neighborhood"), on="Neighborhood")

print(mn_merged.shape)
mn_merged.head()

(69, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Abbey Hey,0.0,0,53.468670,-2.156070
1,Alport Town,0.0,0,53.449186,-2.270495
2,Ancoats,0.0,0,53.485350,-2.229200
3,Ardwick,0.0,0,53.469320,-2.222620
4,Ardwick Green,0.0,0,53.471936,-2.224624


In [72]:
print(mn_merged.shape)
mn_merged.sort_values(["Cluster Labels"], inplace=True)
mn_merged

(69, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Abbey Hey,0.00000,0,53.468670,-2.156070
35,Highfield Country Park,0.00000,0,53.434960,-2.177670
36,Hulme,0.00000,0,53.467270,-2.249730
37,Ladybarn,0.00000,0,53.434650,-2.212480
38,Levenshulme,0.00000,0,53.444850,-2.191590
39,Longsight,0.00000,0,53.456980,-2.200510
40,Manchester city centre,0.00000,0,53.486200,-2.234090
41,Merseybank,0.00000,0,53.422775,-2.262769
42,Miles Platting,0.00000,0,53.495350,-2.211990
43,Moss Side,0.00000,0,53.457460,-2.252370


In [73]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(mn_merged['Latitude'], mn_merged['Longitude'], mn_merged['Neighborhood'], mn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [74]:
map_clusters.save('map_clusters.html')

In [75]:
mn_merged.loc[mn_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Abbey Hey,0.0,0,53.468670,-2.156070
35,Highfield Country Park,0.0,0,53.434960,-2.177670
36,Hulme,0.0,0,53.467270,-2.249730
37,Ladybarn,0.0,0,53.434650,-2.212480
38,Levenshulme,0.0,0,53.444850,-2.191590
39,Longsight,0.0,0,53.456980,-2.200510
40,Manchester city centre,0.0,0,53.486200,-2.234090
41,Merseybank,0.0,0,53.422775,-2.262769
42,Miles Platting,0.0,0,53.495350,-2.211990
43,Moss Side,0.0,0,53.457460,-2.252370


In [76]:
mn_merged.loc[mn_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
60,Spinningfields,0.01,1,53.48107,-2.25316
62,"Strangeways, Manchester",0.01,1,53.47959,-2.24874
13,Brooklands (Manchester ward),0.01,1,53.47959,-2.24874
25,Collyhurst,0.01,1,53.49499,-2.22361


In [77]:
mn_merged.loc[mn_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
59,"Smedley, Manchester",0.02381,2,53.498804,-2.224978
